In [3]:
import numpy as np
import math
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from datetime import date
from jugaad_data.nse import bhavcopy_save, bhavcopy_fo_save
from jugaad_data.nse import stock_df
import pandas_ta as ta

In [4]:
data = stock_df(symbol="SBIN", from_date=date(2023,1,1),to_date=date(2023,10,1), series="EQ")

In [5]:
data

,DATE,SERIES,OPEN,HIGH,LOW,PREV. CLOSE,LTP,CLOSE,VWAP,52W H,52W L,VOLUME,VALUE,NO OF TRADES,SYMBOL
0,2023-09-29,EQ,592.85,600.00,591.30,589.95,598.30,598.55,597.22,629.55,499.35,12189735,7.279947e+09,224164,SBIN
1,2023-09-28,EQ,591.25,595.20,588.85,589.75,590.50,589.95,591.37,629.55,499.35,21960816,1.298696e+10,287488,SBIN
2,2023-09-27,EQ,593.45,594.30,588.20,594.40,590.00,589.75,590.74,629.55,499.35,9865347,5.827827e+09,232626,SBIN
3,2023-09-26,EQ,594.00,595.00,591.25,594.05,594.45,594.40,593.80,629.55,499.35,8327427,4.944809e+09,185873,SBIN
4,2023-09-25,EQ,601.50,601.50,590.45,598.15,593.95,594.05,594.79,629.55,499.35,9712784,5.777110e+09,262285,SBIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,2023-01-06,EQ,607.40,607.45,597.00,605.10,601.00,600.65,601.57,629.55,425.00,7972036,4.795747e+09,154179,SBIN
181,2023-01-05,EQ,607.00,611.60,601.95,605.20,605.75,605.10,605.84,629.55,425.00,8241475,4.992982e+09,146571,SBIN
182,2023-01-04,EQ,611.85,614.95,604.00,612.40,605.60,605.20,607.74,629.55,425.00,7011191,4.260969e+09,113548,SBIN
183,2023-01-03,EQ,614.50,616.60,610.00,612.20,612.95,612.40,613.26,629.55,425.00,6818062,4.181258e+09,122799,SBIN


In [6]:
print(data['SERIES'].unique(),data['SYMBOL'].unique())

['EQ'] ['SBIN']


In [7]:
data.drop(columns=['SERIES','SYMBOL'],inplace=True)


In [8]:
data['NEXT_START'] = data['CLOSE'].shift(-1)

In [9]:
data

,DATE,OPEN,HIGH,LOW,PREV. CLOSE,LTP,CLOSE,VWAP,52W H,52W L,VOLUME,VALUE,NO OF TRADES,NEXT_START
0,2023-09-29,592.85,600.00,591.30,589.95,598.30,598.55,597.22,629.55,499.35,12189735,7.279947e+09,224164,589.95
1,2023-09-28,591.25,595.20,588.85,589.75,590.50,589.95,591.37,629.55,499.35,21960816,1.298696e+10,287488,589.75
2,2023-09-27,593.45,594.30,588.20,594.40,590.00,589.75,590.74,629.55,499.35,9865347,5.827827e+09,232626,594.40
3,2023-09-26,594.00,595.00,591.25,594.05,594.45,594.40,593.80,629.55,499.35,8327427,4.944809e+09,185873,594.05
4,2023-09-25,601.50,601.50,590.45,598.15,593.95,594.05,594.79,629.55,499.35,9712784,5.777110e+09,262285,598.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,2023-01-06,607.40,607.45,597.00,605.10,601.00,600.65,601.57,629.55,425.00,7972036,4.795747e+09,154179,605.10
181,2023-01-05,607.00,611.60,601.95,605.20,605.75,605.10,605.84,629.55,425.00,8241475,4.992982e+09,146571,605.20
182,2023-01-04,611.85,614.95,604.00,612.40,605.60,605.20,607.74,629.55,425.00,7011191,4.260969e+09,113548,612.40
183,2023-01-03,614.50,616.60,610.00,612.20,612.95,612.40,613.26,629.55,425.00,6818062,4.181258e+09,122799,612.20


In [10]:
data['TARGET'] = (data['NEXT_START'] > data['CLOSE']).astype(int)

In [11]:
data

,DATE,OPEN,HIGH,LOW,PREV. CLOSE,LTP,CLOSE,VWAP,52W H,52W L,VOLUME,VALUE,NO OF TRADES,NEXT_START,TARGET
0,2023-09-29,592.85,600.00,591.30,589.95,598.30,598.55,597.22,629.55,499.35,12189735,7.279947e+09,224164,589.95,0
1,2023-09-28,591.25,595.20,588.85,589.75,590.50,589.95,591.37,629.55,499.35,21960816,1.298696e+10,287488,589.75,0
2,2023-09-27,593.45,594.30,588.20,594.40,590.00,589.75,590.74,629.55,499.35,9865347,5.827827e+09,232626,594.40,1
3,2023-09-26,594.00,595.00,591.25,594.05,594.45,594.40,593.80,629.55,499.35,8327427,4.944809e+09,185873,594.05,0
4,2023-09-25,601.50,601.50,590.45,598.15,593.95,594.05,594.79,629.55,499.35,9712784,5.777110e+09,262285,598.15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,2023-01-06,607.40,607.45,597.00,605.10,601.00,600.65,601.57,629.55,425.00,7972036,4.795747e+09,154179,605.10,1
181,2023-01-05,607.00,611.60,601.95,605.20,605.75,605.10,605.84,629.55,425.00,8241475,4.992982e+09,146571,605.20,1
182,2023-01-04,611.85,614.95,604.00,612.40,605.60,605.20,607.74,629.55,425.00,7011191,4.260969e+09,113548,612.40,1
183,2023-01-03,614.50,616.60,610.00,612.20,612.95,612.40,613.26,629.55,425.00,6818062,4.181258e+09,122799,612.20,0


In [12]:
X = data.drop(columns=['TARGET'])
y = data['TARGET']
print(X.shape,y.shape)

(185, 14) (185,)


In [13]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train, y_test = train_test_split(X,y,test_size=0.2)
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(148, 14) (37, 14)
(148,) (37,)


In [14]:
# Dehorty = (Price - Moving Average) / (Moving Standard Deviation * 2)
def get_lorentzian_distance(x1, x2, n_features):
  distance = 0.0
  for i in range(n_features):
    distance += abs(x1[i] - x2[i]) / (1 + (x1[i] - x2[i]) ** 2)
  return distance


In [15]:
model = NearestNeighbors(n_neighbors=10 ,metric=get_lorentzian_distance)

In [18]:
rsi_period = 14  # Set your desired RSI period
cci_period = 14  # Set your desired CCI period
adx_period = 14  # Set your desired ADX period
ndf = pd.DataFrame()

In [19]:

ndf['RSI'] = ta.rsi(data['CLOSE'], length=rsi_period)
ndf['CCI'] = ta.cci(data['HIGH'], data['LOW'], data['CLOSE'], length=cci_period)

In [20]:
adx_df = ta.adx(data['HIGH'], data['LOW'], data['CLOSE'], length=adx_period)
ndf = ndf.join(adx_df,how='inner')

In [21]:
short_sma = data['CLOSE'].rolling(window=5).mean()
long_sma = data['CLOSE'].rolling(window=34).mean()
awesome_oscillator = short_sma - long_sma

In [22]:
wt = awesome_oscillator * ndf['RSI']

In [23]:
wta_period = 9
wta = wt.rolling(window=wta_period).mean()

In [24]:
ndf['WT'] = wt
ndf['WTA'] = wta

In [30]:
ndf

,RSI,CCI,ADX_14,DMP_14,DMN_14,WT,WTA
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
180,63.284041,83.070150,22.899645,22.562040,13.453276,2388.935336,2816.369940
181,64.962649,143.119266,23.327586,23.265944,12.835848,2424.138561,2750.660878
182,65.001372,137.734864,23.929145,23.611053,12.231321,2366.222000,2684.283114
183,67.763851,156.365450,24.589516,23.152763,11.617851,2523.286652,2625.778200


In [29]:
ndf.isnull().sum()

RSI       14
CCI       13
ADX_14    27
DMP_14    14
DMN_14    14
WT        33
WTA       41
dtype: int64